In [1]:
import pandas as pd
import numpy as np
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import SymbolType
from pyecharts.components import Table
import textwrap

In [2]:
# 合并训练集合与测试集合
# 1. 读取数据
headers = [
    '职位名称', '薪资范围',  '地点', 
    '工作经验', '学历要求',  '岗位标签', 
    '公司名称',  '公司类型',  '公司规模', 
    ]

# 2.合并数据
data = pd.read_csv(
    'all.csv',encoding="utf-8",
    header=None,
    names=headers,
   skiprows=1
)
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人
1,数据分析及应用开发实习生,150-200/天,上海-徐汇,无经验,本科,"['Python', 'excel', 'POWER QUERY', 'POWER AUTO...",上海朵安诺电子商务有限公司,民营,20-99人
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人


In [3]:
# 城市数据处理
data['城市'] = data['地点'].apply(lambda x:str(x).split('-')[0])
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海
1,数据分析及应用开发实习生,150-200/天,上海-徐汇,无经验,本科,"['Python', 'excel', 'POWER QUERY', 'POWER AUTO...",上海朵安诺电子商务有限公司,民营,20-99人,上海
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海


In [4]:
def salary_handle(word):
    num=0
    if word[-1] == '万':
        num = float(word.strip('万')) * 10000
    elif word[-1] == '千':
        num = float(word.strip('千')) * 1000
    return num

In [5]:
# 薪资数据处理
# 去除非范围类数据
data = data[data['薪资范围'] != '面议']
data = data[data.薪资范围.str.contains('/天')==False]
data = data[data.薪资范围.str.contains('以下')==False]
data['薪资下限'] = data.薪资范围.apply(lambda x:x.split('-')[0])
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市,薪资下限
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海,1万
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海,7千
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海,6千
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海,1.5万
5,数据分析专员,8千-1.2万,上海-浦东,1-3年,本科,"['SAS', 'SQL']",上汽通用汽车金融,合资,500-999人,上海,8千


In [6]:
data['薪资下限'] = data.薪资下限.apply(lambda x:salary_handle(x))
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市,薪资下限
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海,10000.0
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海,7000.0
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海,6000.0
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海,15000.0
5,数据分析专员,8千-1.2万,上海-浦东,1-3年,本科,"['SAS', 'SQL']",上汽通用汽车金融,合资,500-999人,上海,8000.0


In [7]:
data.loc[data['薪资下限']<=10000.0,'label'] = '<=10k'
data.loc[data['薪资下限']>10000.0,'label'] = '>10k'
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市,薪资下限,label
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海,10000.0,<=10k
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海,7000.0,<=10k
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海,6000.0,<=10k
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海,15000.0,>10k
5,数据分析专员,8千-1.2万,上海-浦东,1-3年,本科,"['SAS', 'SQL']",上汽通用汽车金融,合资,500-999人,上海,8000.0,<=10k


In [11]:
data.loc[data['label'] == '>10k', 'label' ] = 1
data.loc[data['label'] == '<=10k', 'label'] = 0
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市,薪资下限,label
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海,10000.0,0
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海,7000.0,0
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海,6000.0,0
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海,15000.0,1
5,数据分析专员,8千-1.2万,上海-浦东,1-3年,本科,"['SAS', 'SQL']",上汽通用汽车金融,合资,500-999人,上海,8000.0,0


In [13]:
data['工作经验'].value_counts()

1-3年     1469
3-5年     1339
不限       1281
5-10年     513
无经验       117
1年以下       98
10年以上      19
工作经验       14
Name: 工作经验, dtype: int64

In [14]:
data['学历要求'].value_counts()

本科       3073
大专       1257
硕士        398
学历不限       73
高中         16
学历要求       14
博士         11
中专/中技       5
初中及以下       2
中技          1
Name: 学历要求, dtype: int64

In [15]:
data['公司类型'].value_counts()

民营          2342
上市公司         550
国企           514
股份制企业        398
合资           393
外商独资         251
             221
其它           125
事业单位          24
公司类型          14
社会团体           7
港澳台公司          5
律师事务所          2
学校/下级学院        2
代表处            1
医院             1
Name: 公司类型, dtype: int64

In [16]:
data['公司规模'].value_counts()

1000-9999人     1305
20-99人         1050
100-299人       1030
10000人以上        606
500-999人        443
300-499人        266
20人以下           104
公司规模             14
Name: 公司规模, dtype: int64

In [17]:
data['城市'].value_counts()

北京    906
上海    777
深圳    515
广州    477
成都    321
杭州    226
南京    219
武汉    219
郑州    209
西安    202
青岛    197
济南    190
天津    188
重庆    100
厦门     90
地点     14
Name: 城市, dtype: int64

In [34]:
data.drop(columns=["职位名称","薪资范围","地点","岗位标签","公司名称"])

,工作经验,学历要求,公司类型,公司规模,城市,薪资下限,label
0,不限,本科,民营,20-99人,上海,10000.0,0
2,3-5年,本科,民营,20-99人,上海,7000.0,0
3,不限,大专,民营,20-99人,上海,6000.0,0
4,3-5年,本科,民营,100-299人,上海,15000.0,1
5,1-3年,本科,合资,500-999人,上海,8000.0,0
...,...,...,...,...,...,...,...
5137,3-5年,大专,民营,10000人以上,青岛,12000.0,1
5138,不限,大专,民营,20-99人,青岛,5000.0,0
5139,3-5年,本科,民营,300-499人,青岛,6000.0,0
5140,不限,本科,外商独资,500-999人,青岛,15000.0,1


In [35]:
# 确定编码的字段
one_hot_cols = data.columns.tolist()
# 标签的字段不需要编码
one_hot_cols.remove('label')

data_encode = pd.get_dummies(data, columns=one_hot_cols)

# 连续的特征处理
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
# 转换数据类型
data_con = data.astype('str')
# 编码转换
data_encode = data_con.apply(encoder.fit_transform)

print('encode over')
# 数据差分成训练数据，测试数据集。

train = data_encode.loc[0:1000,:]
test = data_encode.loc[1000:, :]

train = train.dropna(axis=0)
test= test.dropna(axis=0)


encode over


In [36]:
from sklearn.svm import SVC

# 准备数据集
X_train = train.drop('label',axis=1)
Y_train =train['label'].astype('int64')  # 转换为数值型

X_test = test.drop('label',axis=1)
Y_test =test['label'].astype('int64')  # 转换为数值型


In [37]:
# model
# LogisticRegression
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(solver='lbfgs',C=1.0,max_iter=10000)
lr_classifier.fit(X_train,Y_train)
train_score = lr_classifier.score(X_train,Y_train)
test_score = lr_classifier.score(X_test,Y_test)
print(F"训练集准确率：{train_score*100:8.2f}% ，测试集准确率：{test_score*100:8.2f}%，逻辑回归")
#逻辑回归
from sklearn.metrics import classification_report
pre_lr = lr_classifier.predict(X_test)
report_lr = classification_report(Y_test,pre_lr)
print(report_lr)
#混淆矩阵
from sklearn.metrics import confusion_matrix
con_lr = confusion_matrix(Y_test,pre_lr)
print(con_lr)

训练集准确率：   89.22% ，测试集准确率：   89.47%，逻辑回归
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      2429
           1       1.00      0.72      0.84      1485

    accuracy                           0.89      3914
   macro avg       0.93      0.86      0.88      3914
weighted avg       0.91      0.89      0.89      3914

[[2427    2]
 [ 410 1075]]


In [39]:
#3. naive bayes
from sklearn.naive_bayes import GaussianNB, BernoulliNB,MultinomialNB
gau_classifier = GaussianNB()
gau_classifier.fit(X_train,Y_train)
train_score = gau_classifier.score(X_train,Y_train)
test_score = gau_classifier.score(X_test,Y_test)
print(F"训练集准确率：{train_score*100:8.2f}% ，测试集准确率：{test_score*100:8.2f}%，朴素贝叶斯分类")
# 朴素贝叶斯分类
from sklearn.metrics import classification_report
pre_gau = gau_classifier.predict(X_test)
report_gau = classification_report(Y_test,pre_gau)
print(report_gau)
#混淆矩阵
from sklearn.metrics import confusion_matrix
con_gau = confusion_matrix(Y_test,pre_gau)
print(con_gau)

训练集准确率：   83.14% ，测试集准确率：   69.21%，朴素贝叶斯分类
              precision    recall  f1-score   support

           0       0.67      0.99      0.80      2429
           1       0.93      0.20      0.33      1485

    accuracy                           0.69      3914
   macro avg       0.80      0.60      0.57      3914
weighted avg       0.77      0.69      0.62      3914

[[2407   22]
 [1183  302]]


In [41]:
#2.随机森林
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train,Y_train)
train_score = rf_classifier.score(X_train,Y_train)
test_score = rf_classifier.score(X_test,Y_test)
print(F"训练集准确率：{train_score*100:8.2f}% ，测试集准确率：{test_score*100:8.2f}%，随机森林")
#随机森林
from sklearn.metrics import classification_report
pre_rf = rf_classifier.predict(X_test)
report_rf = classification_report(Y_test,pre_rf)
print(report_rf)
#混淆矩阵
from sklearn.metrics import confusion_matrix
con_rf = confusion_matrix(Y_test,pre_rf)
print(con_rf)

训练集准确率：  100.00% ，测试集准确率：   98.75%，随机森林
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2429
           1       0.97      1.00      0.98      1485

    accuracy                           0.99      3914
   macro avg       0.98      0.99      0.99      3914
weighted avg       0.99      0.99      0.99      3914

[[2387   42]
 [   7 1478]]
